In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import importlib
import sys

In [6]:
if "../" not in sys.path:
    sys.path.append("../")
import appUtilities as au

In [7]:
df = pd.read_pickle('../data/sample_data.pkl')
df.Date=pd.to_datetime(df.Date)

In [8]:
df.head()

,Date,Item,Type,Subtype,Location,Payment Method,Amount,ShopBack?,Notes
0,2023-03-17,Prawn Noodles,Dining,Lunch,Work,PayLah,7.5,No,NaN
1,2023-03-17,Poke Bowl,Dining,Dinner,Novena,GrabPay,14.5,Yes,NaN
2,2023-03-17,Bread,Dining,Breakfast,Barcook,PayLah,3.0,No,NaN
3,2023-03-18,Car Wash,Transport,Private Transport,SPC Novena,Cash,12.0,No,NaN
4,2023-03-18,Thunder Tea Rice,Dining,Lunch,Vivo,GrabPay,16.0,Yes,YZ
